# Programming Binary Logistic Regression
You can access videos of the derivation of Logistic Regression here: https://youtu.be/FGnoHdjFrJ8.


In [ ]:
# or play it in the browser right here! 
from IPython.display import YouTubeVideo
YouTubeVideo("FGnoHdjFrJ8")

___
The objective function that we defined was (if we convert to take mean, rather than the sum):


$$ l(\mathbf{w}) = \frac{1}{M}\cdot\sum_i \left( y^{(i)} \ln [g(\mathbf{w}^T \mathbf{x}^{(i)})] + (1-y^{(i)}) (\ln [1 - g(\mathbf{w}^T \mathbf{x}^{(i)})])  \right)  $$

From the videos, the main result is the *update equation* for **binary logistic regression**. The update equation for logistic regression can be written generically as:

$$ w_j \leftarrow w_j + \eta \nabla l(\mathbf{w}) $$

And more specifically as:
$$ \underbrace{w_j}_{\text{new value}} \leftarrow \underbrace{w_j}_{\text{old value}} + \frac{\eta}{M} \underbrace{\sum_{i=1}^M (y^{(i)}-g(\mathbf{w}^T\mathbf{x}^{(i)}))x^{(i)}_j}_{\text{gradient}} $$

where $\eta$ is the learning rate, $g(\cdot)$ is the sigmoid function, $y^{(i)}$ is an instance of the target, $\hat{y}^{(i)}$ is an instance of the prediction (for the current value of $w$), and $x^{(i)}_j$ is the $i^{th}$ instance of the input with the $j^{th}$ feature (that is, the feature in row $i$ and column $j$). The process is iterative, so we must calculate the gradient every time $w$ is updated. We derived this for the simple gradient ascent approach, where we calculate the gradient and step in the direction of steepest ascent. 

We can simplify this calculation a bit using matrix algebra to solve for the entire vector $w$:

$$ \mathbf{w} \leftarrow \mathbf{w} + \frac{\eta}{M} \sum_{i=1}^M (y^{(i)}-g(\mathbf{w}^T\mathbf{x}^{(i)}))\cdot\mathbf{x}^{(i)} $$

where we simply multiply the entire $\mathbf{x}^{(i)}$ instance by the scalar difference, $y^{(i)}-\widehat{y}^{(i)}$. 

___



Now lets start to program logistic regression in the binary case. Recall that the logistic regression equation directly regresses the logistic function (also called a sigmoid) for the probability $y=1$. In this case,

$$ p(y^{(i)}=1\text{ | }\mathbf{x}^{(i)},\mathbf{w})=g(\mathbf{w}^T \mathbf{x}^{(i)})=\frac{1}{1+\exp{(-\mathbf{w}^T \mathbf{x}^{(i)}})}$$

Recall also that the dot product of $w^T x^{(i)}$ results in a scalar value. That is, we multiply $w$ and $x^{(i)}$ together to get a single value, then place that single value through the sigmoid to get the probability that $y=1$. The prediction can be written as:

$$ \widehat{y}^{(i)} = 1 \text{      if,  } g(\mathbf{w}^T \mathbf{x}^{(i)}) > 0.5  $$ and $$ \widehat{y}^{(i)} = 0 \text{      otherwise  } $$

___
Let's start by programming the sigmoid and predictions. 

In [ ]:
import numpy as np
class BinaryLogisticRegressionBase:
    # private:
    def __init__(self, eta, iterations=20):
        self.eta = eta
        self.iters = iterations
        # internally we will store the weights as self.w_ to keep with sklearn conventions
    
    def __str__(self):
        return 'Base Binary Logistic Regression Object, Not Trainable'
    
    # convenience, private and static:
    @staticmethod
    def _sigmoid(theta):
        return 1/(1+np.exp(-theta)) 
    
    @staticmethod
    def _add_intercept(X):
        return np.hstack((np.ones((X.shape[0],1)),X)) # add bias term
    
    # public:
    def predict_proba(self, X, add_intercept=True):
        # add bias term if requested
        Xb = self._add_intercept(X) if add_intercept else X
        return self._sigmoid(Xb @ self.w_) # return the probability y=1
    
    def predict(self,X):
        return (self.predict_proba(X)>0.5) #return the actual prediction
    
    
        
blr = BinaryLogisticRegressionBase(0.1)
print(blr)


___

Now let's add the model training function, using the update equation defined previously:

$$ \mathbf{w} \leftarrow \mathbf{w} + \eta \frac{1}{M}\sum_{i=1}^M (y^{(i)}-g(\mathbf{w}^T\mathbf{x}^{(i)}))\mathbf{x}^{(i)} $$

In [ ]:
# inherit from base class
class BinaryLogisticRegression(BinaryLogisticRegressionBase):
    #private:
    def __str__(self):
        if(hasattr(self,'w_')):
            return 'Binary Logistic Regression Object with coefficients:\n'+ str(self.w_) # is we have trained the object
        else:
            return 'Untrained Binary Logistic Regression Object'


    @property
    def coef_(self):
        if(hasattr(self,'w_')):
            return self.w_[1:]
        else:
            return None

    @property
    def intercept_(self):
        if(hasattr(self,'w_')):
            return self.w_[0]
        else:
            return None

        
    def _get_gradient(self,X,y):
        # programming \sum_i (yi-g(xi))xi
        gradient = np.zeros(self.w_.shape) # set gradient to zero
        for (xi,yi) in zip(X,y):
            # the actual update inside of sum
            gradi = (yi - self.predict_proba(xi,add_intercept=False))*xi 
            # reshape to be column vector and add to gradient
            gradient += gradi.reshape(self.w_.shape) 
        
        return gradient/float(len(y))
       
    # public:
    def fit(self, X, y):
        Xb = self._add_intercept(X) # add bias term
        num_samples, num_features = Xb.shape
        
        self.w_ = np.zeros((num_features,1)) # init weight vector to zeros
        
        # for as many as the max iterations
        for _ in range(self.iters):
            gradient = self._get_gradient(Xb,y)
            self.w_ += gradient*self.eta # multiply by learning rate 

            
blr = BinaryLogisticRegression(0.1)
print(blr)

To understand how our classifier works, we need some data. Let's first lets load in some example data, the iris dataset.

Notice that we have made the iris classification problem **binary**. Instead of three classes, we have made it only two classes. Most values are zero (as seen in the pie chart above). While only about a third of all instances are of class type "1".

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import numpy as np
import plotly

ds = load_iris()
X = ds.data
y = (ds.target>1).astype(int) # make problem binary

X_train, X_test, y_train, y_test = train_test_split(X,y, train_size = 0.8, test_size=0.2)

plotly.offline.init_notebook_mode() # run at the start of every notebook

graph1 = {'x': np.unique(y),
          'y': np.bincount(y)/len(y),
            'type': 'bar'}

fig = dict()
fig['data'] = [graph1]
fig['layout'] = {'title': 'Binary Class Distribution',
                'autosize':False,
                'width':400,
                'height':400}

plotly.offline.iplot(fig)

In [ ]:
%%time
# Now we can train the classifier
blr = BinaryLogisticRegression(eta=0.1,iterations=12)
blr.fit(X_train,y_train)
print(blr)

In [ ]:
print("coef",blr.coef_)
print("intercept",blr.intercept_)
print("w",blr.w_)

___


### Self Test Question
What part of the equation is represented by this piece of code:

    self.predict_proba(xi)
    
$$ \mathbf{w} \leftarrow \mathbf{w} + \eta \frac{1}{M}\sum_{i=1}^M \underbrace{(\underbrace{y^{(i)}}_{\bf A}-\underbrace{g(\mathbf{w}^T \mathbf{x}^{(i)})}_{\bf B})}_{\bf D}\underbrace{\mathbf{x}^{(i)}}_{\bf C} $$

___

So how well did the training go?

In [ ]:
from sklearn.metrics import accuracy_score

yhat = blr.predict(X_test)
print('Accuracy of: ',accuracy_score(y_test,yhat))

In [ ]:
%%time
# can we do better? Maybe more iterations?
params = dict(eta=0.01,
              iterations=500)

blr = BinaryLogisticRegression(**params)
blr.fit(X_train,y_train)
print(blr)
yhat = blr.predict(X_test)
print('Accuracy of: ',accuracy_score(y_test,yhat))

# Extending to Vectorized Programming
Excellent! We programmed the correct update equation. But, it seems to be really slow. That's because python is not fast with `for` loops. We can speed up the process using vectorization in numpy. This is fast because numpy is written in c++ under the hood. Let's change our update equation to use only linear algebra and fewer summations (which are for loops).  

Lets start with only using $X$, instead of $x^{(i)}$. $X$ is the matrix of all instances in each row: 

$$ \mathbf{X}=\begin{bmatrix}
        \leftarrow &  \mathbf{x}^{(1)}      & \rightarrow  \\
        \leftarrow &  \mathbf{x}^{(2)}      & \rightarrow  \\
        &  \vdots &\\
        \leftarrow &  \mathbf{x}^{(M)}      & \rightarrow  \\
     \end{bmatrix}
$$

Let's also create a matrix of the differences between the target and prediction:
$$ \mathbf{y}_{diff}=\begin{bmatrix}
        y^{(1)}-g(\mathbf{w}^T\mathbf{x}^{(1)})  \\
        y^{(2)}-g(\mathbf{w}^T\mathbf{x}^{(2)})  \\
        \vdots \\
        y^{(M)}-g(\mathbf{w}^T\mathbf{x}^{(M)})  \\
     \end{bmatrix}=\begin{bmatrix}
        y_{diff}^{(1)}  \\
        y_{diff}^{(2)}  \\
        \vdots \\
        y_{diff}^{(M)}  \\
     \end{bmatrix}
$$

What we really want to do is scale each row of $\mathbf{X}$ by the corresponding scalar value of $\mathbf{y}_{diff}$. Luckily for us, this is exactly how `numpy` interprets multiplication of a matrix with with a column vector that has the same number of rows. So we can simply write 

$$\nabla l(\mathbf{w}) =\frac{1}{M}\sum_{i=1}^M (y^{(i)}-g(\mathbf{w}^T\mathbf{x}^{(i)}))\mathbf{x}^{(i)}$$

as

$$\nabla l(\mathbf{w}) = \text{mean}(\mathbf{y}_{diff}\odot\mathbf{X})_{columns}$$

where $\odot$ is a row-wise operator that multiplies the scalar value in each row of $\mathbf{y}_{diff}$ to each element in the same row as $\mathbf{X}$.

$$ \mathbf{y}_{diff}\odot\mathbf{X}=\begin{bmatrix}
        y_{diff}^{(1)}  \\
        y_{diff}^{(2)}  \\
        \vdots \\
        y_{diff}^{(M)}  \\
     \end{bmatrix}\cdot
\begin{bmatrix}
        \leftarrow &  \mathbf{x}^{(1)}      & \rightarrow  \\
        \leftarrow &  \mathbf{x}^{(2)}      & \rightarrow  \\
        &  \vdots &\\
        \leftarrow &  \mathbf{x}^{(M)}      & \rightarrow  \\
     \end{bmatrix}=\begin{bmatrix}
        (\leftarrow &  \mathbf{x}^{(1)}      & \rightarrow) \cdot y_{diff}^{(1)} \\
        (\leftarrow &  \mathbf{x}^{(2)}      & \rightarrow) \cdot y_{diff}^{(2)} \\
        &  \vdots &\\
        (\leftarrow &  \mathbf{x}^{(M)}      & \rightarrow) \cdot y_{diff}^{(M)} \\
        \end{bmatrix}
$$

In [ ]:
%%time
# now lets do some vectorized coding
import numpy as np
from scipy.special import expit

class VectorBinaryLogisticRegression(BinaryLogisticRegression):
    # inherit from our previous class to get same functionality
    @staticmethod
    def _sigmoid(theta):
        # increase stability, redefine sigmoid operation
        return expit(theta) #1/(1+np.exp(-theta))
    
    # but overwrite the gradient calculation
    def _get_gradient(self,X,y):
        ydiff = y-self.predict_proba(X,add_intercept=False).ravel() # get y difference
        gradient = np.mean(X * ydiff[:,np.newaxis], axis=0) # make ydiff a column vector and multiply through
        
        return gradient.reshape(self.w_.shape)

# use same params as defined above
blr = VectorBinaryLogisticRegression(**params)
blr.fit(X_train,y_train)
print(blr.w_)
yhat = blr.predict(X_test)
print('Accuracy of: ',accuracy_score(y_test,yhat))



### Self Test Question: 
If we had a class imbalance problem, with one type of class dominating the response, how could we manipulate the above `fit` method to give more importance to a given class?
- **A.** Add weights to the gradient calculation (for each instance of ydiff)
- **B.** Repeat certain classes in the update of the gradient calculation
- **C.** Update objective function to only look at performance on the worst performing class


___

# Programming Multiclass Logistic Regression
Our previous methods only worked for a single class, to extend this beyond binary classification we can use a common trick: train a separate Binary Classifier for each unique class. During prediction, we run an instance through each Binary Classifier, choosing the Binary Classifier that has the highest probability. **This is a method known as one-versus-all** because we only assume one class is positive when we train each separate Binary Classifier. 

This graphic breaks down the process:
- graphic from: https://antonhaugen.medium.com/introducing-mllibs-one-vs-rest-classifier-402eeab22493

<img src="https://miro.medium.com/max/1400/1*4Ii3aorSLU50RV6V5xalzg.png" alt="OVA" width="500" >

To program this, its surprisingly simple now that we have a fast binary classifier. 

In [ ]:
class LogisticRegression:
    def __init__(self, eta, iterations=20):
        self.eta = eta
        self.iters = iterations
        # internally we will store the weights as self.w_ to keep with sklearn conventions
    
    def __str__(self):
        if(hasattr(self,'w_')):
            return 'MultiClass Logistic Regression Object with coefficients:\n'+ str(self.w_) # is we have trained the object
        else:
            return 'Untrained MultiClass Logistic Regression Object'

    @property
    def coef_(self):
        if(hasattr(self,'w_')):
            return self.w_[:,1:]
        else:
            return None

    @property
    def intercept_(self):
        if(hasattr(self,'w_')):
            return self.w_[:,0]
        else:
            return None
        
    def fit(self,X,y):
        num_samples, num_features = X.shape
        self.unique_ = np.unique(y) # get each unique class value
        num_unique_classes = len(self.unique_)
        self.classifiers_ = [] # will fill this array with binary classifiers
        
        for i,yval in enumerate(self.unique_): # for each unique value
            y_binary = (y==yval) # create a binary problem
            # train the binary classifier for this class
            blr = VectorBinaryLogisticRegression(self.eta,
                                                 self.iters)
            blr.fit(X,y_binary)
            # add the trained classifier to the list
            self.classifiers_.append(blr)
            
        # save all the weights into one matrix, separate column for each class
        self.w_ = np.hstack([x.w_ for x in self.classifiers_]).T
        
    def predict_proba(self,X):
        probs = []
        for blr in self.classifiers_:
            probs.append(blr.predict_proba(X)) # get probability for each classifier
        
        return np.hstack(probs) # make into single matrix
    
    def predict(self,X):
        return self.unique_[np.argmax(self.predict_proba(X),axis=1)] # take argmax along row
    
lr = LogisticRegression(0.1,1500)
print(lr)

In [ ]:
%%time
ds = load_iris()
X = ds.data
y = ds.target # note problem is NOT binary anymore, there are three classes!

X_train, X_test, y_train, y_test = train_test_split(X,y, train_size = 0.8, test_size=0.2)


lr = LogisticRegression(0.1,500)
lr.fit(X_train,y_train)
print(lr)

yhat = lr.predict(X_test)
print('Accuracy of: ',accuracy_score(y_test,yhat))

In [ ]:
print("coef\n",lr.coef_)
print("intercept\n",lr.intercept_)
print("w\n",lr.w_)

# Comparing to Scikit Learn
Of course, there was no **need** to program the above except for the learning experience (or because we want to extend the learning algorithm without adjusting the scikit-learn source code). Now, let's use the builtin API for `sklearn`. 

In [ ]:
%%time
from sklearn.linear_model import LogisticRegression as SKLogisticRegression

lr_sk = SKLogisticRegression(solver='liblinear') # all params default

lr_sk.fit(X_train,y_train)
print(np.hstack((lr_sk.intercept_[:,np.newaxis],lr_sk.coef_)))
yhat = lr_sk.predict(X_test)
print('Accuracy of: ',accuracy_score(y_test,yhat))

Why is this so much faster? The answer is parallelization is better handled through sklearn, separating out each classification, and the operations are ALL in lower level c++. **The main reason, however, is that  the update used is more advanced than what we have talked about.** They have an array of solvers that use the gradient (and Hessian) to solve the optimization more quickly. The accuracy is higher because this logistic regression object uses regularization, which helps control for over-learning, and also because the optimization allows for finer steps when we get to places where the gradient is small and the objective function is less well behaved. 
____
# Logistic Regression Class Boundaries

In [ ]:
# linear boundaries visualization from sklearn documentation
from matplotlib import pyplot as plt
import copy
%matplotlib inline
plt.style.use('ggplot')

def plot_decision_boundaries(lr,Xin,y,title=''):
    Xb = copy.deepcopy(Xin)
    lr.fit(Xb[:,:2],y) # train only on two features

    h=0.01
    # create a mesh to plot in
    x_min, x_max = Xb[:, 0].min() - 1, Xb[:, 0].max() + 1
    y_min, y_max = Xb[:, 1].min() - 1, Xb[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))

    # get prediction values
    Z = lr.predict(np.c_[xx.ravel(), yy.ravel()])

    # Put the result into a color plot
    Z = Z.reshape(xx.shape)
    plt.contourf(xx, yy, Z, cmap=plt.cm.Paired, alpha=0.5)

    # Plot also the training points
    plt.scatter(Xb[:, 0], Xb[:, 1], c=y, cmap=plt.cm.Paired)
    plt.xlabel('Sepal length')
    plt.ylabel('Sepal width')
    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())
    plt.xticks(())
    plt.yticks(())
    plt.title(title)
    plt.show()
    
lr = LogisticRegression(0.1,1500) # this is still OUR LR implementation, not sklearn
plot_decision_boundaries(lr,X,y)

# L2 Regularized Logistic Regression
Can we control these boundaries?
Whenever we let the weights become large, that means we are allowing the sigmoid finction to also become more vertical (see image below). Large values of $w$ correspond to the blue line. Smaller values of $w$ correspond to the red line. So, in order to help prevent overfitting, we can add in a term into our optimization that incentivizes the weights to be smaller in magnitude to help prevent the sigmoid from having a large slope.

<img src="https://www.mathworks.com/matlabcentral/mlc-downloads/downloads/submissions/51007/versions/2/screenshot.jpg" alt="Drawing" width= 300/>



In [ ]:
from ipywidgets import widgets as wd

def sig_explorer(w):
    x = np.linspace(-10,10,100)
    y = expit(w*x)
    plt.plot(x,y)
    plt.xlabel("x")
    plt.ylabel("g(x)")
    plt.show()

wd.interact(sig_explorer, w=(0.5,5,0.5),__manual=True)

So keeping the weights small is identical to having smaller multipliers in the sigmoid function. Let's introduce another parameter in the objective function called the L2-Norm. The idea here is that we want to keep the values of the weights relatively small. This helps to control overfitting. We are changing our objective function by adding in a new summation of the weights that reduces the maximum value when weights are large:

$$ l(\mathbf{w})_{reg} = \underbrace{l(\mathbf{w})_{prev}}_{\text{old objective}} - \underbrace{C\cdot\sum_j w_j^2}_{\text{regularization}} $$

$C$ is a hyperparameter controlling how much we want to optimize the previous objective function versus keep the weights small. This means the gradient will be updated as follows:

$$ \underbrace{w_j}_{\text{new value}} \leftarrow \underbrace{w_j}_{\text{previous value}} + \eta \underbrace{\left[\left(\sum_{i=1}^M (y^{(i)}-g(\mathbf{w}^T\mathbf{x}^{(i)}))x^{(i)}_j\right) - C \cdot 2w_j \right]}_{\text{new gradient}}$$

Which is the same as:
$$ \mathbf{w} \leftarrow \mathbf{w} + \eta \left[\underbrace{\nabla l(\mathbf{w})_{prev}}_{\text{previous gradient}} - C \cdot 2\mathbf{w} \right]$$

Its fairly easy to implement because we just need to add to the existing gradient function.

In [ ]:
class RegularizedBinaryLogisticRegression(VectorBinaryLogisticRegression):
    # extend init functions
    def __init__(self, C=0.0, **kwds):        
        # need to add to the original initializer 
        self.C = C
        # but keep other keywords
        super().__init__(**kwds) # call parent initializer
        
        
    # extend previous class to change functionality
    def _get_gradient(self,X,y):
        # call get gradient from previous class
        gradient = super()._get_gradient(X,y)
        
        # add in regularization (to all except bias term)
        gradient[1:] += -2 * self.w_[1:] * self.C
        return gradient
        

In [ ]:
# now redefine the Logistic Regression Function where needed
class RegularizedLogisticRegression(LogisticRegression):
    def __init__(self, C=0.0, **kwds):        
        # need to add to the original initializer 
        self.C = C
        # but keep other keywords
        super().__init__(**kwds) # call parent initializer
        
    def fit(self,X,y):
        num_samples, num_features = X.shape
        self.unique_ = np.unique(y) # get each unique class value
        num_unique_classes = len(self.unique_)
        self.classifiers_ = [] # will fill this array with binary classifiers
        
        for i,yval in enumerate(self.unique_): # for each unique value
            y_binary = y==yval # create a binary problem
            # train the binary classifier for this class
            # now this has regularization built into it
            blr = RegularizedBinaryLogisticRegression(eta=self.eta,
                                                      iterations=self.iters,
                                                      C=self.C)
            blr.fit(X,y_binary)
            # add the trained classifier to the list
            self.classifiers_.append(blr)
            
        # save all the weights into one matrix, separate column for each class
        self.w_ = np.hstack([x.w_ for x in self.classifiers_]).T

In [ ]:
from ipywidgets import widgets as wd

# create a possible value of C from 0.001 to 0.1
cost_vals = np.logspace(-3,1,15)
def lr_explor(cost_idx):
    C = cost_vals[cost_idx]
    lr_clf = RegularizedLogisticRegression(eta=0.1,
                                           iterations=2500, # lots of iterations (to help overfit)
                                           C=C) # get object
    
    plot_decision_boundaries(lr_clf,X,y,title="C=%.5f"%(C))
    plt.show()

wd.interact(lr_explor,cost_idx=(0,14,1),__manual=True)

# what happens when C gets too large??
# let's explore the different values and see what happens

___
# Next Time: More Optimization with LR
___

In this notebook you learned:
- Formulation of Logistic regression 
- Vectorizing Code to run Faster (in C++ via Numpy)
- Regularization using L2-Norm of weights
- Multi-Class Logistic Regression
- LR Class Boundaries (showed they are linear) 